In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import time
from reference import get_file_names

In [4]:
df_ref = pd.read_csv('./vidas_data/cross_ref_cols_tabs.csv')
# df_ref.head()

In [5]:
get_file_names(['IDEHR', 'chk_stato_danimo'], df_ref)

['wH_DirectTable_VISITA_MED_VIDAS', 'wH_DirectTable_C_VISITA_MED_VIDAS']